In [1]:
import numpy as np

import tfmindi as tm

In [2]:
motif_collection_dir = tm.fetch_motif_collection()
motif_annotations_file = tm.fetch_motif_annotations()

In [3]:
motif_collection = tm.load_motif_collection(motif_collection_dir)
motif_annotations = tm.load_motif_annotations(motif_annotations_file)

In [4]:
motif_to_db = tm.load_motif_to_dbd(motif_annotations)  # dict of motif to dbd

In [5]:
oh_path = "/home/lukas/projects/TF-MInDi/tests/data/sample_oh.npz"
contrib_path = "/home/lukas/projects/TF-MInDi/tests/data/sample_contrib.npz"

oh = np.load(oh_path)["oh"]
contrib = np.load(contrib_path)["contrib"]

In [6]:
seqlets_df, seqlets_matrix = tm.pp.extract_seqlets(contrib=contrib, oh=oh, threshold=0.05, additional_flanks=3)

Processing seqlets: 100%|██████████| 295/295 [00:00<00:00, 19381.57it/s]


In [7]:
seqlets_df.head()  # Display the first few rows of the extracted seqlets DataFrame

,example_idx,start,end,attribution,p-value
0,24,59,76,-0.467904,1.887379e-15
1,31,178,198,1.294061,2.467105e-04
2,28,299,318,-0.457312,7.206342e-04
3,24,306,324,0.948738,7.408627e-04
4,32,455,480,1.100183,1.234263e-03


In [8]:
print(seqlets_matrix[0].shape)
print(len(seqlets_matrix))

(4, 17)
295


In [9]:
sim_matrix = tm.pp.calculate_motif_similarity(seqlets_matrix, motif_collection)

In [10]:
sim_matrix.shape

(295, 17995)

In [11]:
# Test with full new storage approach including motif annotations
adata = tm.pp.create_seqlet_adata(
    sim_matrix,
    seqlets_df,
    seqlet_matrices=seqlets_matrix,
    oh_sequences=oh,
    contrib_scores=contrib,
    motif_collection=motif_collection,
    motif_annotations=motif_annotations,
    motif_to_dbd=motif_to_db,
)
print(f"AnnData shape: {adata.shape}")
print(f"AnnData X type: {type(adata.X)}")
print(f"AnnData obs columns: {list(adata.obs.columns)}")
print(f"AnnData var columns: {list(adata.var.columns)}")
print(f"Number of motifs with DBD annotations: {adata.var['dbd'].notna().sum()}")
print(f"Number of motifs with Direct annotations: {adata.var['Direct_annot'].notna().sum()}")
print(f"Example DBD values: {adata.var['dbd'].dropna().head().tolist()}")
adata

AnnData shape: (295, 17995)
AnnData X type: <class 'numpy.ndarray'>
AnnData obs columns: ['example_idx', 'start', 'end', 'attribution', 'p-value', 'seqlet_matrix', 'seqlet_oh', 'example_oh', 'example_contrib']
AnnData var columns: ['motif_pwm', 'Direct_annot', 'Motif_similarity_annot', 'Orthology_annot', 'Motif_similarity_and_Orthology_annot', 'dbd']
Number of motifs with DBD annotations: 2964
Number of motifs with Direct annotations: 2374
Example DBD values: ['Homeodomain', 'p53', 'Paired box', 'bZIP', 'Homeodomain']


AnnData object with n_obs × n_vars = 295 × 17995
    obs: 'example_idx', 'start', 'end', 'attribution', 'p-value', 'seqlet_matrix', 'seqlet_oh', 'example_oh', 'example_contrib'
    var: 'motif_pwm', 'Direct_annot', 'Motif_similarity_annot', 'Orthology_annot', 'Motif_similarity_and_Orthology_annot', 'dbd'

In [12]:
adata.obs

,example_idx,start,end,attribution,p-value,seqlet_matrix,seqlet_oh,example_oh,example_contrib
0,24,59,76,-0.467904,1.887379e-15,"[[-0.0011337847152989144, 0.039029417622873375...","[[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...","[[0.00199191365391016, 0.00013025905354879797,..."
1,31,178,198,1.294061,2.467105e-04,"[[0.22378262176327485, 0.02587442114950032, -0...","[[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.001131125376559794, -0.000327722518704831..."
2,28,299,318,-0.457312,7.206342e-04,"[[0.09934498645434618, -0.09970337874610243, 0...","[[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[-0.0005060430848971009, -0.00117746787145733..."
3,24,306,324,0.948738,7.408627e-04,"[[-0.013875630033374423, -0.005492392312432093...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...","[[0.00199191365391016, 0.00013025905354879797,..."
4,32,455,480,1.100183,1.234263e-03,"[[-0.03798202052374473, 0.08325835300999583, -...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,...","[[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[-0.0009635935421101749, 0.004236725158989429..."
...,...,...,...,...,...,...,...,...,...
290,34,270,280,-0.065786,4.950383e-02,"[[0.022109211680040108, 0.7850121193320718, 0....","[[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,...","[[5.0170550821349025e-05, 9.499986481387168e-0..."
291,31,354,364,-0.072184,4.950383e-02,"[[0.1615160816741255, 0.5031887579013183, 1.0,...","[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.001131125376559794, -0.000327722518704831..."
292,32,235,245,0.161017,4.964185e-02,"[[-0.40209289486089034, -0.5591923998092229, 0...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[-0.0009635935421101749, 0.004236725158989429..."
293,30,89,99,0.336421,4.964185e-02,"[[0.007080417389271871, 0.018909457498695617, ...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...","[[-0.00010426474182168022, 0.00324180908501148..."


In [13]:
tm.tl.cluster_seqlets(adata, resolution=3.0)

Computing PCA...
Computing neighborhood graph...


/home/lukas/.local/share/hatch/env/virtual/tfmindi/EarZ-jtt/docs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Computing t-SNE embedding...
Performing Leiden clustering with resolution 3.0...
Calculating mean contribution scores...
Assigning DBD annotations...
Computing consensus DBD annotations per cluster...
Clustering complete. Found 20 clusters.
DBD annotation coverage: 234/295 seqlets


In [27]:
adata.obs

,example_idx,start,end,attribution,p-value,seqlet_matrix,seqlet_oh,example_oh,example_contrib,leiden,mean_contrib,seqlet_dbd,cluster_dbd
0,24,59,76,-0.467904,1.887379e-15,"[[-0.0011337847152989144, 0.039029417622873375...","[[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...","[[0.00199191365391016, 0.00013025905354879797,...",0,0.201815,None,NaN
1,31,178,198,1.294061,2.467105e-04,"[[0.22378262176327485, 0.02587442114950032, -0...","[[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.001131125376559794, -0.000327722518704831...",1,0.212963,None,NaN
2,28,299,318,-0.457312,7.206342e-04,"[[0.09934498645434618, -0.09970337874610243, 0...","[[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[-0.0005060430848971009, -0.00117746787145733...",2,0.170461,None,C2H2 ZF; Homeodomain
3,24,306,324,0.948738,7.408627e-04,"[[-0.013875630033374423, -0.005492392312432093...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...","[[0.00199191365391016, 0.00013025905354879797,...",3,0.183521,None,NaN
4,32,455,480,1.100183,1.234263e-03,"[[-0.03798202052374473, 0.08325835300999583, -...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,...","[[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[-0.0009635935421101749, 0.004236725158989429...",1,0.237585,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,34,270,280,-0.065786,4.950383e-02,"[[0.022109211680040108, 0.7850121193320718, 0....","[[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,...","[[5.0170550821349025e-05, 9.499986481387168e-0...",15,0.352965,Homeodomain,ARID/BRIGHT
291,31,354,364,-0.072184,4.950383e-02,"[[0.1615160816741255, 0.5031887579013183, 1.0,...","[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.001131125376559794, -0.000327722518704831...",16,0.270725,None,Ets
292,32,235,245,0.161017,4.964185e-02,"[[-0.40209289486089034, -0.5591923998092229, 0...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[-0.0009635935421101749, 0.004236725158989429...",11,0.353924,None,Homeodomain
293,30,89,99,0.336421,4.964185e-02,"[[0.007080417389271871, 0.018909457498695617, ...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...","[[-0.00010426474182168022, 0.00324180908501148...",1,0.248367,None,NaN
